## 데이터 시각화

### 관련 라이브러리 호출

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### 작업 경로 확인 및 변경

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 작업 경로에 있는 폴더명과 파일명을 출력합니다.
sorted(os.listdir())

### 실습 데이터셋 준비

In [ ]:
# 'Seafood_Trade_Prep.z' 파일을 호출합니다.
# [힌트] joblib.load() 함수를 사용하세요.
# [참고] 두 데이터프레임을 원소로 갖는 리스트를 반환합니다.
joblib.load(filename = 'Seafood_Trade_Prep.z')

In [ ]:
# 압축 파일을 호출하고 sfd, mgd에 할당합니다.
sfd, mgd = joblib.load(filename = 'Seafood_Trade_Prep.z')

In [ ]:
# sfd의 정보를 확인합니다.
sfd.info()

In [ ]:
# sfd의 처음 5행을 출력합니다.
sfd.head()

In [ ]:
# mgd의 정보를 확인합니다.
mgd.info()

In [ ]:
# mgd의 처음 5행을 출력합니다.
mgd.head()

### 그래픽 파라미터 설정

In [ ]:
# 그래픽 파라미터를 설정합니다.
# [주의] 해당 폰트를 먼저 설치해야 합니다.
plt.rc(group = 'font', family = 'Gowun Dodum', size = 8)
plt.rc(group = 'figure', figsize = (4, 4), dpi = 150)
plt.rc(group = 'axes', unicode_minus = False)
plt.rc(group = 'legend', frameon = True, fc = '1', ec = '0')

### 히스토그램 시각화

In [ ]:
# mgd의 국민소득에 대한 기술통계량을 확인합니다.
# [참고] 히스토그램을 그리기 전에 연속형 변수의 최솟값과 최댓값을 확인합니다.
mgd['국민소득'].describe()

In [ ]:
# mgd의 국민소득로 히스토그램을 그립니다.
# [힌트] sns.histplot() 함수를 사용하세요.
# [참고] binrange와 binwidth 매개변수를 추가하면 계급을 지정할 수 있습니다.
# [참고] fc(채우기색) 및 ec(테두리색) 매개변수를 추가하면 색을 지정할 수 있습니다.
sns.histplot(data = mgd, x = '국민소득', binrange = (0, 100000), binwidth = 10000,
             fc = 'silver', ec = 'black')

# 그래프 제목을 '국민소득 분포'로 지정합니다.
plt.title(label = '국민소득 분포');

### 국가별 수산물품목별 무역수지를 반환하는 함수 생성

In [ ]:
# 국가명을 지정하면 수산물품목별 무역수지를 반환하는 함수를 생성합니다.
def getMpritmList(nation):
    
    # sfd에서 nation에 해당하는 행을 df에 할당합니다.
    df = sfd[sfd['nation'].eq(nation)]
    
    # df로 피벗 테이블을 생성하고 mpr에 할당합니다.
    mpr = pd.pivot_table(
        data = df, 
        values = 'grand', 
        index = 'mpritm', 
        columns = 'imxprt', 
        aggfunc = np.sum
    )
    
    # mpr에 있는 결측값을 0으로 대체하고 mpr에 재할당합니다.
    mpr = mpr.fillna(value = 0)
    
    # mpr의 행이름 네임을 '수산물'로 변경합니다.
    mpr.index.name = '수산물'
    
    # mpr의 열이름 네임을 삭제합니다.
    mpr.columns.name = None
    
    # mpr의 수출에서 수입을 뺀 무역수지를 생성합니다.
    mpr['무역수지'] = mpr['수출'] - mpr['수입']
    
    # mpr을 무역수지로 내림차순 정렬하고 mpr에 재할당합니다.
    mpr = mpr.sort_values(by = ['무역수지'], ascending = False)
    
    # mpr의 행이름을 초기화하고 mpr에 재할당합니다.
    mpr = mpr.reset_index()

    # 결과를 반환합니다.
    return mpr

In [ ]:
# 일본의 수산물품목별 무역수지를 계산하고 mpr에 할당합니다.
mpr = getMpritmList(nation = '일본')

In [ ]:
# mpr의 처음 10행을 출력합니다.
mpr.head(n = 10)

In [ ]:
# mpr의 마지막 10행을 출력합니다.
mpr.tail(n = 10)

### 막대그래프 그리기

In [ ]:
# mpr의 처음 10행을 top10에 할당합니다.
top10 = mpr.head(n = 10)

# top10으로 막대그래프를 그립니다
# [힌트] sns.barplot() 함수를 사용하세요.
# [참고] x 매개변수에 범주형, y 매개변수에 연속형 변수를 지정합니다.
sns.barplot(data = top10, x = '수산물', y = '무역수지', palette = 'Spectral')

# 그래프 제목을 '수출 상위 10위 수산물 목록'으로 지정합니다.
plt.title(label = '수출 상위 10위 수산물 목록')

# x축 눈금명을 90도 회전시킵니다.
plt.xticks(rotation = 90)

# 막대 위에 무역수지를 반올림하여 정수만 남긴 수를 텍스트로 추가합니다.
for i, v in enumerate(top10['무역수지']):
    plt.text(x = i, y = v, s = int(round(v, 0)), ha = 'center', va = 'bottom', fontsize = 8);

### 선 그래프 그리기

In [ ]:
# sfd의 month별 수출/수입 합계를 선 그래프로 그립니다.
# [힌트] sns.lineplot() 함수를 사용하세요.
sns.lineplot(data = sfd, x = 'month', y = 'grand', hue = 'imxprt', errorbar = None)

# 그래프 제목을 '월별 수출입 금액 합계'로 지정합니다.
plt.title(label = '월별 수출입 금액 합계')

# x축 눈금명을 90도 회전시킵니다.
plt.xticks(rotation = 90);

### 산점도 그리기

In [ ]:
# mgd의 국민소득과 수출로 산점도를 그립니다.
# [힌트] sns.scatterplot() 함수를 사용하세요.
# [참고] hue 매개변수에 범주형 변수명을 지정하면 점의 채우기 색을 다르게 지정합니다.
# [참고] 점의 크기는 s 매개변수에 정수로 지정합니다.(기본값: 50)
sns.scatterplot(data = mgd, x = '국민소득', y = '수출', hue = '국가등급')

# 그래프 제목을 '국민소득과 수출의 관계'로 지정합니다.
plt.title(label = '국민소득과 수출의 관계')

# 범례의 위치를 오른쪽 위로 지정합니다.
plt.legend(loc = 'upper right');

## End of Document